In [1]:
from config import *
import torch
from models_local import ResidualBlock
from models_local import Model
import torch.nn as nn
import torch.optim as optim

from OFDM_SDR_Functions_torch import *
import numpy as np
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.pdsch_iq = []
        self.pilot_iq = []
        self.labels = []
        
    def __len__(self):
        return len(self.pdsch_iq)
    
    def __getitem__(self, index):
        x1 = self.pdsch_iq[index]
        x2 = self.pilot_iq[index]
        y = self.labels[index]
        return x1, x2, y
    
    def add_item(self, new_pdsch_iq, new_pilot_iq, new_label):
        self.pdsch_iq.append(new_pdsch_iq)
        self.pilot_iq.append(new_pilot_iq)
        self.labels.append(new_label)
        
dataset = CustomDataset()

In [3]:
dataset = torch.load('ofdm_dataset.pth')

In [4]:
model = Model(S, F, Qm)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [7]:
#make a training loop
num_epochs = 100

data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Create an instance of the Rx1Model
model = Model(S, F, Qm)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0

    for pilot_iq, pdsch_iq, labels in data_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(pilot_iq, pdsch_iq)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

    # Print average loss for the epoch
    average_loss = total_loss / len(data_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")

# Save the trained model if needed
torch.save(model.state_dict(), 'rx1_model.pth')


AttributeError: 'CustomDataset' object has no attribute 'data'

In [ ]:
training_loops = 100

#symbol_index = 1
#tx_gain_i=0
#rx_gain=0
# Here channel simulations are used for training the model

ch_SINR_min = 20 # channel emulation min SINR
ch_SINR_max = 20 # channel emulation max SINR

for i in range(number_of_testcases):
    if i % 100 == 0:
        print(i)
    ch_SINR = int(random.uniform(ch_SINR_min, ch_SINR_max)) # SINR generation
    pdsch_bits, TX_Samples = create_OFDM_data() # data stream
    RX_Samples = generate_cdl_c_impulse_response(tx_signal = TX_Samples, num_samples=100, sampling_rate=SampleRate, SINR=ch_SINR, repeats = 2, random_start=False) # convolve channel with tx signal
    #symbol_index=sync_TTI(TX_Samples, RX_Samples, leading_zeros)
    #SINR_meas, noise_power, signal_power = SINR(RX_Samples, 400, symbol_index)
    RX_NO_CP = CP_removal(RX_Samples, 0, S, FFT_size, CP, plotsig=False)
    RX_NO_CP = RX_NO_CP / torch.max(torch.abs(RX_NO_CP))
    OFDM_demod = DFT(RX_NO_CP, plotDFT=False)
    
    
    # H_estim = channelEstimate_LS(TTI_mask_RE, pilot_symbols, F, FFT_offset, Sp, OFDM_demod, plotEst=False)
    # OFDM_demod_no_offsets = remove_fft_Offests(OFDM_demod, F, FFT_offset)
    # equalized_H_estim = equalize_ZF(OFDM_demod_no_offsets, H_estim, F, S)
    # QAM_est = get_payload_symbols(TTI_mask_RE, equalized_H_estim, FFT_offset, F, plotQAM=False)
    # PS_est, hardDecision = Demapping(QAM_est, de_mapping_table_Qm)
    # bits_est = PS(PS_est)
    # error_count = torch.sum(bits_est != pdsch_bits.flatten()).float()  # Count of unequal bits
    # error_rate = error_count / bits_est.numel()  # Error rate calculation
    # BER = round(error_rate.item(),3)  # Round to 3 decimal places
    # # SINR2BER_table[i,0] = round(tx_gain_i,1)
    # SINR2BER_table[i,1] = round(rx_gain,1)
    # SINR2BER_table[i,2] = round(SINR_meas,1)
    # SINR2BER_table[i,3] = round(BER,1)
    dataset.add_item(OFDM_demod, pdsch_bits)
    
torch.save(dataset, 'ofdm_dataset.pth')
    

NameError: name 'number_of_testcases' is not defined